<a href="https://colab.research.google.com/github/anushka-code/Code-Smell-Classification/blob/main/Code_Smell_Classification_using_Deep_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Code Smell Detection (Smelly/Non Smelly) using Structural & Semantic Features 

#Code Smells Targetted: 


1.   Long Parameters List
2.   Switch Statements







###Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import tensorflow
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Input, Convolution1D, MaxPooling1D, Flatten, Dense, Dropout
from keras.utils import np_utils
import sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import RepeatedKFold 
from sklearn import model_selection
import imblearn
from imblearn.over_sampling import SMOTE
import nltk 
from nltk.tokenize import RegexpTokenizer
import re
import gensim
from gensim.models import Word2Vec
from operator import length_hint

###Mounting Google Drive

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
    
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

###Dataset Loader

In [3]:
def DataLoader(link, name_of_file):
  id = link.split("/")[-2]
  downloaded = drive.CreateFile({'id':id}) 
  downloaded.GetContentFile(name_of_file)
  dataframe = pd.read_csv(name_of_file)
  return dataframe

link1 = 'https://drive.google.com/file/d/1EfbAqgr7i9h4yFwEoU3igG34Gt48l6WT/view?usp=sharing'
link2 = 'https://drive.google.com/file/d/1Ya1OMWsz1yyXAaZheIck-roX0M9UWiqg/view?usp=sharing'
link3 = 'https://drive.google.com/file/d/1xeMRB6ACyjep6VF_rDRq8cXkA58ZlEZs/view?usp=sharing'
link4 = 'https://drive.google.com/file/d/1OxxMX7-kc7mQUvb2DSJR_fbtvZ5bB1Ef/view?usp=sharing'
name1 = 'long_parameters_list_structural.csv'
name2 = 'switch_statements_structural.csv'
name3 = 'long_parameters_list_semantic.csv'
name4 = 'switch_statements_semantic.csv'

df_lp = DataLoader(link1, name1)
df_ss = DataLoader(link2, name2)
df_lpm = DataLoader(link3, name3)
df_ssm = DataLoader(link4, name4)

Long Parameters List Structural Dataset

In [4]:
df_lp.head(5)

,NOP_method,CC_method,ATFD_method,FDP_method,CM_method,MAXNESTING_method,LOC_method,CYCLO_method,NMCS_method,NOLV_method,...,NOI_package,LOC_package,NOM_package,NOPK_project,NOCS_project,NOI_project,NOM_project,NOMNAMM_project,LOC_project,is_long_parameters_list
0,0,0,0,0,0,2,8,2,0,0,...,0,229,16,43,849,78,5788,4778,65687,False
1,0,0,0,0,0,1,3,1,0,0,...,0,4223,240,43,849,78,5788,4778,65687,False
2,1,0,0,0,0,1,2,1,0,1,...,0,4223,240,43,849,78,5788,4778,65687,False
3,0,5,0,0,9,0,1,1,0,0,...,11,6037,728,43,849,78,5788,4778,65687,False
4,1,0,1,1,0,1,3,1,0,1,...,11,6037,728,43,849,78,5788,4778,65687,False


Switch Statements Structural Dataset



In [5]:
df_ss.head(5)

,NOP_method,CC_method,ATFD_method,FDP_method,CM_method,MAXNESTING_method,LOC_method,CYCLO_method,NMCS_method,NOLV_method,...,NOI_package,LOC_package,NOM_package,NOPK_project,NOCS_project,NOI_project,NOM_project,NOMNAMM_project,LOC_project,is_switch_statements
0,1,2,0,0,2,1,5,1,0,1,...,3,439,53,43,849,78,5788,4778,65687,False
1,0,0,0,0,0,2,6,3,0,0,...,0,4223,240,43,849,78,5788,4778,65687,False
2,2,0,0,0,0,1,3,1,0,2,...,0,4223,240,43,849,78,5788,4778,65687,False
3,0,0,0,0,0,2,3,1,0,0,...,0,1212,120,43,849,78,5788,4778,65687,False
4,0,0,0,0,0,1,3,1,0,0,...,0,516,29,43,849,78,5788,4778,65687,False


Long Parameters List Semantic Dataset

In [6]:
df_lpm.head(5)

,project,package,class,method,is_code_smell
0,/mnt/qualitas/qualitas_corpus_compile/freemind...,freemind.modes.common.plugins,ReminderHookBase,shutdownMapHook(),False
1,/mnt/qualitas/qualitas_corpus_compile/freemind...,freemind.modes.mindmapmode,MindMapMapModel.StringReaderCreator,createReader(),False
2,/mnt/qualitas/qualitas_corpus_compile/freemind...,freemind.modes.mindmapmode,MindMapController,newModel(ModeController),False
3,/mnt/qualitas/qualitas_corpus_compile/freemind...,freemind.modes,MindMapNode,getColor(),False
4,/mnt/qualitas/qualitas_corpus_compile/freemind...,freemind.modes,ControllerAdapter.ControllerPopupMenuListener,popupMenuCanceled(PopupMenuEvent),False


Switch Statements Semantic Dataset

In [7]:
df_ssm.head(5)

,project,package,class,method,is_code_smell
0,/mnt/qualitas/qualitas_corpus_compile/freemind...,freemind.modes.mindmapmode.actions.xml,AbstractXmlAction,actionPerformed(ActionEvent),False
1,/mnt/qualitas/qualitas_corpus_compile/freemind...,freemind.modes.mindmapmode,EncryptedMindMapNode,childrenUnfolded(),False
2,/mnt/qualitas/qualitas_corpus_compile/freemind...,freemind.modes.mindmapmode,freemind.modes.mindmapmode.MindMapController$1,"apply(MindMapMapModel,MindMapNodeModel)",False
3,/mnt/qualitas/qualitas_corpus_compile/freemind...,freemind.view.mindmapview.attributeview,freemind.view.mindmapview.attributeview.Attrib...,run(),False
4,/mnt/qualitas/qualitas_corpus_compile/freemind...,freemind.modes.mindmapmode.listeners,MindMapNodeMotionListener,stopDrag(),False


# Data Pre-Processing of Structural Dataset


In [8]:
def PrePro(last_column, dataframe):
  dataframe.rename(columns = {last_column :'is_code_smell'}, inplace = True) #rename column
  dataframe['is_code_smell'] = dataframe["is_code_smell"].astype(int) #change boolean labels to int labels
  Y_part = dataframe.iloc[:,-1:]
  X_part = dataframe.iloc[:,:56]
  X_part = X_part.replace(to_replace =["?"], value = np.nan) #replace non existing values with null
  X_part = X_part.astype(float) #change datatype of features of X as float
  return X_part,Y_part

X_lp, Y_lp = PrePro('is_long_parameters_list',df_lp)
X_ss, Y_ss = PrePro('is_switch_statements',df_ss)

In [9]:
X_lp

,NOP_method,CC_method,ATFD_method,FDP_method,CM_method,MAXNESTING_method,LOC_method,CYCLO_method,NMCS_method,NOLV_method,...,NOMNAMM_package,NOI_package,LOC_package,NOM_package,NOPK_project,NOCS_project,NOI_project,NOM_project,NOMNAMM_project,LOC_project
0,0.0,0.0,0.0,0.0,0.0,2.0,8.0,2.0,0.0,0.0,...,9.0,0.0,229.0,16.0,43.0,849.0,78.0,5788.0,4778.0,65687.0
1,0.0,0.0,0.0,0.0,0.0,1.0,3.0,1.0,0.0,0.0,...,198.0,0.0,4223.0,240.0,43.0,849.0,78.0,5788.0,4778.0,65687.0
2,1.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,0.0,1.0,...,198.0,0.0,4223.0,240.0,43.0,849.0,78.0,5788.0,4778.0,65687.0
3,0.0,5.0,0.0,0.0,9.0,0.0,1.0,1.0,0.0,0.0,...,523.0,11.0,6037.0,728.0,43.0,849.0,78.0,5788.0,4778.0,65687.0
4,1.0,0.0,1.0,1.0,0.0,1.0,3.0,1.0,0.0,1.0,...,523.0,11.0,6037.0,728.0,43.0,849.0,78.0,5788.0,4778.0,65687.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415,7.0,1.0,2.0,2.0,1.0,3.0,55.0,5.0,0.0,7.0,...,223.0,3.0,9478.0,312.0,42.0,1171.0,66.0,10384.0,8750.0,312068.0
416,14.0,0.0,0.0,0.0,0.0,1.0,18.0,1.0,0.0,14.0,...,190.0,0.0,8318.0,262.0,42.0,1171.0,66.0,10384.0,8750.0,312068.0
417,8.0,1.0,3.0,1.0,1.0,2.0,16.0,6.0,0.0,8.0,...,190.0,0.0,8318.0,262.0,42.0,1171.0,66.0,10384.0,8750.0,312068.0
418,11.0,1.0,0.0,0.0,1.0,1.0,11.0,1.0,0.0,11.0,...,190.0,0.0,8318.0,262.0,42.0,1171.0,66.0,10384.0,8750.0,312068.0


In [10]:
def MeanforNaN(dataframe):   #function to fill null spaces with column mean 
  column_means = dataframe.mean()
  dataframe = dataframe.fillna(column_means)
  return dataframe

X_lp = MeanforNaN(X_lp)
X_ss = MeanforNaN(X_ss)

In [11]:
def ConCat(df1,df2): #concatenate code smell datasets
  code_smells = [df1,df2]
  joint = pd.concat(code_smells)
  return joint

X_train = ConCat(X_lp,X_ss)
Y_train = ConCat(Y_lp,Y_ss)

###Data Normalization

In [12]:
def Normalize(dataframe): #apply MinMax normalisation to fit the values between 0 to 1
  scaler = MinMaxScaler()
  model = scaler.fit(dataframe)
  scaled_data = model.transform(dataframe)
  return scaled_data
X_sample = Normalize(X_train)

###Synthetic Minority Oversampling Technique (SMOTE) Algo for Imbalanced Datasets


In [13]:
Y_train.value_counts() #1/3rd are smelly, 2/3rd are non smelly 

is_code_smell
0                573
1                267
dtype: int64

In [14]:
Y_sample = Y_train.to_numpy(dtype='int64', copy='True')

In [15]:
def Oversample(X_data,Y_data): # Using Smote obtain a 50-50 balanced dataset 

  sm = SMOTE(random_state = 2)
  X_train_res, Y_train_res = sm.fit_resample(X_data, Y_data.ravel())
  return X_train_res, Y_train_res

X_new, Y_new = Oversample(X_sample,Y_sample)

In [16]:
X_train_final = X_new.reshape((1146,56,1)) # hence makes 573 smelly and 573 non smelly samples

###Repeated K-Fold Cross Validation for Limited Data


In [17]:
def CrossValidation(dataX,dataY): # split the dataset into train and test using k-5 fold cross validation
  kf = RepeatedKFold(n_splits=5, n_repeats=10, random_state=None) 

  for train_index, test_index in kf.split(dataX):
        X_crosstrain, X_crosstest = dataX[train_index], dataX[test_index] 
        Y_crosstrain, Y_crosstest = dataY[train_index], dataY[test_index]
        return X_crosstrain,X_crosstest,Y_crosstrain,Y_crosstest

X_crosstrain,X_crosstest,Y_crosstrain,Y_crosstest = CrossValidation(X_train_final, Y_new)

In [18]:
X_crosstrain.shape #916 samples for train

(916, 56, 1)

In [19]:
X_crosstest.shape #230 samples for test

(230, 56, 1)

# Text Pre-Processing of Semantic Dataset

In [20]:
df_sem = ConCat(df_lpm,df_ssm) #join long parameters list and switch statements dataset

In [21]:
def PrePro2(dataframe):
  Y_part = dataframe.iloc[:,-1:]
  X_part = dataframe.iloc[:,:4]  
  return X_part, Y_part

X_sem , Y_sem = PrePro2(df_sem) #separate as X and Y columns

###Tokenization of the groups : project, package, class, method

In [22]:
def ToList(dataframe, string): #converts pandas.core.series.Series to list of lists
  new_column = dataframe[string].tolist()
  return list(map(lambda x:[x], new_column))

project_words = ToList(X_sem, 'project')
package_words = ToList(X_sem, 'package')
class_words = ToList(X_sem, 'class')
method_words = ToList(X_sem, 'method')

In [23]:
def TokenizeGroups(text): #tokenizes as per CamelCase RegEx and converts to lowercase
  tokenizer = RegexpTokenizer('[a-zA-Z](?:[a-z]+|[A-Z]*(?=[A-Z]|$))') # RegEx pattern for CamelCase 
  useful_text = tokenizer.tokenize(text) 
  useful_text = [x.lower() for x in useful_text]
  return useful_text

In [24]:
def LoopOver(list): #loops over all the samples to tokenize all the strings in each word group
  for i in range(0,840):
    list[i] = TokenizeGroups(list[i][0])
    i=i+1
  return list

project_words_sem = LoopOver(project_words)
package_words_sem = LoopOver(package_words)
class_words_sem = LoopOver(class_words)
method_words_sem = LoopOver(method_words)

In [25]:
def WordList(list1,list2,list3,list4): #concatenate the words of each project, package, class and method
  final_list = []
  for i in range(0,840):
    x = list1[i] + list2[i] + list3[i] + list4[i]
    final_list.append(x)
  return final_list

word_groups = WordList(project_words_sem,package_words_sem, class_words_sem, method_words_sem) #get an 840 sampled list of words

In [26]:
print(word_groups)

[['mnt', 'qualitas', 'qualitas', 'corpus', 'compile', 'freemind', 'freemind', 'modes', 'common', 'plugins', 'reminder', 'hook', 'base', 'shutdown', 'map', 'hook'], ['mnt', 'qualitas', 'qualitas', 'corpus', 'compile', 'freemind', 'freemind', 'modes', 'mindmapmode', 'mind', 'map', 'map', 'model', 'string', 'reader', 'creator', 'create', 'reader'], ['mnt', 'qualitas', 'qualitas', 'corpus', 'compile', 'freemind', 'freemind', 'modes', 'mindmapmode', 'mind', 'map', 'controller', 'new', 'model', 'mode', 'controller'], ['mnt', 'qualitas', 'qualitas', 'corpus', 'compile', 'freemind', 'freemind', 'modes', 'mind', 'map', 'node', 'get', 'color'], ['mnt', 'qualitas', 'qualitas', 'corpus', 'compile', 'freemind', 'freemind', 'modes', 'controller', 'adapter', 'controller', 'popup', 'menu', 'listener', 'popup', 'menu', 'canceled', 'popup', 'menu', 'event'], ['mnt', 'qualitas', 'qualitas', 'corpus', 'compile', 'freemind', 'freemind', 'modes', 'mind', 'map', 'link', 'registry', 'get', 'link', 'for', 'i',

In [32]:
len(word_groups)

840

#Word2Vec using Gensim

In [31]:
def WordEmbedding(tokenizedwords): #creates a dictionary with vocab words and corresponding vectors 
  model = Word2Vec(tokenizedwords, min_count=1, sg=1) #skipgram model 
  model.train(tokenizedwords, total_examples=model.corpus_count, epochs=model.epochs)
  keys = list(model.wv.vocab.keys())
  values = list(model.wv.vectors)
  wdict = {}
  for i in range(840):
    wdict[keys[i]] = values[i]
  return wdict

wordvec = WordEmbedding(word_groups)

#Positional Embedding

# Neural Network Architecture

### Left Branch : CNN 




In [28]:
def CNNFeatureExtraction(trainX): # CNN with 3 conv1d layers for feature extraction from structural feature
  epochs, batch_size, test_batch_size = 120, 50, 50
  height, width, depth = trainX.shape[0], trainX.shape[1], trainX.shape[2]
  model_left = Sequential()
  model_left.add(Convolution1D(filters=32, kernel_size=3, activation='relu', input_shape=(width,depth)))
  model_left.add(Convolution1D(filters=64, kernel_size=3, activation='relu'))
  model_left.add(Convolution1D(filters=64, kernel_size=3, activation='relu'))
  model_left.add(Flatten())
  return model_left

### Right Branch : Transformer Encoder

In [29]:
# def evaluate_model(trainX, trainy, testX, testy): 
# 	epochs, batch_size, test_batch_size = 120, 50, 50
# 	height, width, depth = trainX.shape[0], trainX.shape[1], trainX.shape[2]
# 	model = Sequential()
# 	model.add(Convolution1D(filters=32, kernel_size=3, activation='relu', input_shape=(width,depth)))
# 	model.add(Convolution1D(filters=64, kernel_size=3, activation='relu'))
# 	model.add(Flatten())
# 	model.add(Dense(1, activation='sigmoid'))
# 	model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# 	model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size)
# 	_, accuracy = model.evaluate(testX, testy, batch_size=test_batch_size)
# 	return accuracy

# model_accuracy = evaluate_model(X_crosstrain, Y_crosstrain, X_crosstest, Y_crosstest)